In [62]:
import QuantLib as ql
import utils 
%matplotlib inline
from QuantLib import *
import datetime
import re
import pandas as pd
import QuantLib as ql
import pandas as pd
import numpy as np
import yfinance as yf
from yahoo_fin.stock_info import get_quote_table
from pylab import plt
plt.style.use('dark_background')

In [64]:
import datetime
import re
import pandas as pd
import QuantLib as ql
import pandas as pd
import numpy as np
import yfinance as yf
from yahoo_fin.stock_info import get_quote_table
from pylab import plt
plt.style.use('dark_background')
import pandas as pd
def options_chain(symbol):
    info = get_quote_table(symbol)
    current_price = info["Quote Price"]
    tk = yf.Ticker(symbol)
    exps = tk.options
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)
    options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
    options['yte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['midpoint'] = (options['bid'] + options['ask']) / 2 
    options['spread'] =  (options['ask'] - options['bid'])
    options['spread_pct'] = (options['ask'] - options['bid'])/options['ask'] 
    options['Underlying'] = current_price 
    options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
    return pd.DataFrame({
            "Underlying": current_price,
            "Ticker": symbol,
            "Expiry": options["expirationDate"],
            "YTE": options["yte"],
            "DTE": options["dte"],
            "Call": options["CALL"],
            "IV": options["impliedVolatility"],        
            "Strike": options["strike"],
            "Last": options["lastPrice"],
            "Bid": options["bid"],
            "Ask": options["ask"],
            "Midpoint": options['midpoint'],
            "Spread": options['spread'],
            "Spread_Pct": options['spread_pct'],
            "AKA": options["contractSymbol"]})

def filter_by_moneyness(df, pct_cutoff=0.2):
    crit1 = (1-pct_cutoff)*df.Strike < df.Underlying
    crit2 = df.Underlying< (1+pct_cutoff)*df.Strike
    return (df.loc[crit1 & crit2].reset_index(drop=True))

In [65]:
df=options_chain("AAPL")
df=filter_by_moneyness(df)

In [66]:
def ql_option(ticker, option="Call"):
    
    def create_option(row):
        today = ql.Date.todaysDate()

                
        underlying = ql.SimpleQuote(current_price)
        dividendYield = ql.FlatForward(today, dividend_yield, ql.Actual360())

        exercise = EuropeanExercise(ql.Date(expiration.day, expiration.month, expiration.year))
        payoff = PlainVanillaPayoff(ql.Option.Call, row["strike"])
        option = EuropeanOption(payoff,exercise)
                
        S = QuoteHandle(SimpleQuote(current_price))
        q = YieldTermStructureHandle(FlatForward(today, dividend_yield, Actual360()))
        r = YieldTermStructureHandle(FlatForward(0, TARGET(), 0.03, Actual360()))
        
        sigma = BlackVolTermStructureHandle(BlackConstantVol(0, TARGET(), row["impliedVolatility"]*100, Actual360()))
        process = BlackScholesMertonProcess(S,q,r,sigma)  

        engine = AnalyticEuropeanEngine(process)
        option.setPricingEngine(engine)
        return {"Underlying": current_price,
                "Ticker": ticker,
                "Strike": row["strike"],
                "Last": row["lastPrice"],
                "Bid": row["bid"],
                "Ask": row["ask"],
                "IV": row["impliedVolatility"],
                "NPV": option.NPV(),
                "Delta": option.delta(),
                "Gamma": option.gamma(),
                "Theta": option.theta() / 365,
                "AKA": row["contractSymbol"]}
    
    info = get_quote_table(ticker)
    current_price = info["Quote Price"]
    yield_re = re.compile(r"\((?P<value>(\d+\.\d+))%\)")
    try:
        dividend_yield = float(yield_re.search(info["Forward Dividend & Yield"])["value"])
    except (KeyError, ValueError, TypeError):
        dividend_yield = 0.0   

    options_= pd.DataFrame()   
        
    if option == "Call":
        
        calls = pd.DataFrame()
        
        tk = yf.Ticker(ticker)
        exps = tk.options

        for e in exps:
            opt = tk.option_chain(e)
            calls = pd.DataFrame().append(opt.calls)
            calls['expirationDate'] = e
            expiration =  pd.to_datetime(e) + datetime.timedelta(days = 1)
            calls['expirationDate'] = expiration
            calls['yte'] = (calls['expirationDate'] - datetime.datetime.today()).dt.days / 365
            calls['dte'] = (calls['expirationDate'] - datetime.datetime.today()).dt.days
            calls[['bid', 'ask', 'strike']] = calls[['bid', 'ask', 'strike']].apply(pd.to_numeric)
            calls = calls.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate'])
            options = calls.apply(create_option, axis=1, result_type="expand")
            options_ = options_.append(options, ignore_index=True) 
    else:    
        print("None")
    return  options_


def filter_by_moneyness(df, pct_cutoff=0.2):
    crit1 = (1-pct_cutoff)*df.Strike < df.Underlying
    crit2 = df.Underlying< (1+pct_cutoff)*df.Strike
    return (df.loc[crit1 & crit2].reset_index(drop=True))


In [67]:
filter_by_moneyness(ql_option("AAPL", option="Call"))

,Underlying,Ticker,Strike,Last,Bid,Ask,IV,NPV,Delta,Gamma,Theta,AKA
0,120.529999,AAPL,101.25,19.35,0.0,0.0,0.000010,19.060983,0.998113,0.000000,0.215804,AAPL210319C00101250
1,120.529999,AAPL,102.50,19.32,0.0,0.0,0.000010,17.811087,0.998113,0.000000,0.215701,AAPL210319C00102500
2,120.529999,AAPL,103.75,16.95,0.0,0.0,0.000010,16.561191,0.998113,0.000000,0.215599,AAPL210319C00103750
3,120.529999,AAPL,105.00,15.80,0.0,0.0,0.000010,15.311296,0.998113,0.000000,0.215496,AAPL210319C00105000
4,120.529999,AAPL,106.25,15.55,0.0,0.0,0.000010,14.061400,0.998113,0.000000,0.215393,AAPL210319C00106250
...,...,...,...,...,...,...,...,...,...,...,...,...
420,120.529999,AAPL,130.00,20.44,0.0,0.0,0.015635,15.860339,0.173307,0.000333,0.022282,AAPL230317C00130000
421,120.529999,AAPL,135.00,19.00,0.0,0.0,0.015635,15.670446,0.171784,0.000336,0.021807,AAPL230317C00135000
422,120.529999,AAPL,140.00,17.35,0.0,0.0,0.031260,28.827598,0.244998,0.000031,0.048889,AAPL230317C00140000
423,120.529999,AAPL,145.00,16.20,0.0,0.0,0.031260,28.802808,0.244866,0.000032,0.048768,AAPL230317C00145000


Example

Vanilla Call Option


In [1]:
from QuantLib import *

exercise = EuropeanExercise(Date(22,August,2022))
payoff = PlainVanillaPayoff(Option.Call, 100.0)
option = EuropeanOption(payoff,exercise)

S = QuoteHandle(SimpleQuote(100.0))
r = YieldTermStructureHandle(FlatForward(0, TARGET(), 0.03, Actual360()))
q = YieldTermStructureHandle(FlatForward(0, TARGET(), 0.01, Actual360()))
sigma = BlackVolTermStructureHandle(BlackConstantVol(0, TARGET(), 0.20, Actual360()))
process = BlackScholesMertonProcess(S,q,r,sigma)



option.impliedVolatility(11.10, process)

#engine = AnalyticEuropeanEngine(process)
#option.setPricingEngine(engine)
#option.NPV()
#option.delta()

0.20704721302072543